In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split


import tensorflow as tf
from keras.models import Sequential,load_model
from keras import layers,Input
from keras.losses import binary_crossentropy
from keras.layers import Dense
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
import keras.backend as K

## Multiqubit

In [61]:
Q0state0=np.load('../Data/new/2gsps/Q0state0_her.npy')
Q0state1=np.load('../Data/new/2gsps/Q0state1_her.npy')
Q0output0=np.zeros((Q0state0.shape[0]))
Q0output1=np.ones((Q0state1.shape[0]))
print(Q0state0.shape, Q0state1.shape)

Q1state0=np.load('../Data/new/2gsps/Q1state0_her.npy')
Q1state1=np.load('../Data/new/2gsps/Q1state1_her.npy')
indices = np.arange(Q1state1.shape[0])
np.random.shuffle(indices)
selected_indices = indices[:950]
Q1state0 = Q1state0[selected_indices]
Q1state1 = Q1state1[selected_indices]
Q1output0=np.zeros((Q1state0.shape[0]))
Q1output1=np.ones((Q1state1.shape[0]))
print(Q1state0.shape, Q1state1.shape)

(967, 8192, 2) (948, 8192, 2)
(950, 8192, 2) (950, 8192, 2)


In [62]:
st,ed=700,2700
Q0state0=np.mean(Q0state0[:,st:ed,:], axis=1)
Q0state1=np.mean(Q0state1[:,st:ed,:], axis=1)
Q1state0=np.mean(Q1state0[:,st:ed,:], axis=1)
Q1state1=np.mean(Q1state1[:,st:ed,:], axis=1)
print(Q0state0.shape, Q0state1.shape, Q1state0.shape, Q1state1.shape)

(967, 2) (948, 2) (950, 2) (950, 2)


In [63]:
print(Q0state0,Q1state0)
Q0state0=np.hstack((Q0state0,np.zeros((Q0state0.shape[0],1))))
Q0state1=np.hstack((Q0state1,np.zeros((Q0state1.shape[0],1))))
Q1state0=np.hstack((Q1state0,np.ones((Q1state0.shape[0],1))))
Q1state1=np.hstack((Q1state1,np.ones((Q1state1.shape[0],1))))
print(Q0state0,Q1state0)
print(Q0state0.shape, Q0state1.shape, Q1state0.shape, Q1state1.shape)

[[-1.86258963 -3.4491148 ]
 [ 1.51223662 -5.54383595]
 [-1.98299879 -2.33745642]
 ...
 [-1.43970027 -3.84779876]
 [ 5.49199819 -0.16619367]
 [ 1.83827626 -3.7038209 ]] [[-0.67298194 -4.71165805]
 [ 0.15761759 -3.31193289]
 [-2.21697453 -3.67511811]
 ...
 [-3.73552051 -3.89711095]
 [-1.53497018  0.57580245]
 [-1.46468201 -3.70130676]]
[[-1.86258963 -3.4491148   0.        ]
 [ 1.51223662 -5.54383595  0.        ]
 [-1.98299879 -2.33745642  0.        ]
 ...
 [-1.43970027 -3.84779876  0.        ]
 [ 5.49199819 -0.16619367  0.        ]
 [ 1.83827626 -3.7038209   0.        ]] [[-0.67298194 -4.71165805  1.        ]
 [ 0.15761759 -3.31193289  1.        ]
 [-2.21697453 -3.67511811  1.        ]
 ...
 [-3.73552051 -3.89711095  1.        ]
 [-1.53497018  0.57580245  1.        ]
 [-1.46468201 -3.70130676  1.        ]]
(967, 3) (948, 3) (950, 3) (950, 3)


In [64]:
x=np.vstack((Q0state0,Q0state1,Q1state0,Q1state1))
print(x.shape)
y=np.hstack((Q0output0,Q0output1,Q1output0,Q1output1))
print(y.shape)

(3815, 3)
(3815,)


In [65]:

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=45)
print(x_train.shape)
print(y_train.sum())

(3052, 3)
1523.0


In [66]:
# x_trainC=x_train.reshape((x_train.shape[0],x_train.shape[1]*2))
# x_testC=x_test.reshape((x_test.shape[0],x_test.shape[1]*2))
classifier = LogisticRegression(random_state = 42, max_iter=500)
classifier.fit(x_train, y_train)
print(classifier.score(x_test,y_test))
pred = classifier.predict(x_test)
count=0
count1=0
for i in range(pred.shape[0]):
    if y_test[i]==0:
        count+=1
        if pred[i]<=0.5:
            count1+=1
print(count,count1)
count=0
count1=0
for i in range(pred.shape[0]):
    if y_test[i]==1:
        count+=1
        if pred[i]>=0.5:
            count1+=1
print(count,count1)

0.8532110091743119
388 314
375 337


In [67]:
model=SVC(decision_function_shape='ovo')
model.fit(x_train,y_train)
print(model.score(x_test,y_test))
pred=model.predict(x_test)
count=0
count1=0
for i in range(pred.shape[0]):
    if y_test[i]==0:
        count+=1
        if pred[i]<=0.5:
            count1+=1
print(count,count1)
count=0
count1=0
for i in range(pred.shape[0]):
    if y_test[i]==1:
        count+=1
        if pred[i]>=0.5:
            count1+=1
print(count,count1)

0.8885976408912188
388 338
375 340


In [96]:
n_components = 2
gmm = GaussianMixture(n_components=n_components)
gmm.fit(x_train)
y_pred = gmm.predict(x_test)
print(gmm.score)
count=0
count1=0
for i in range(y_pred.shape[0]):
    if y_test[i]==0:
        count+=1
        if y_pred[i]<=0.5:
            count1+=1
print(count,count1)
count=0
count1=0
for i in range(y_pred.shape[0]):
    if y_test[i]==1:
        count+=1
        if y_pred[i]>=0.5:
            count1+=1
print(count,count1)

0.9257627517729033
388 0
375 372


In [84]:
def arch():
    model=Sequential()
    model.add(Input(shape=(3), name='Input Layer'))
    model.add(Dense(8,activation='relu', name='HiddenLayer1'))
    model.add(Dense(4,activation='relu', name='HiddenLayer2'))
    model.add(Dense(1,activation='sigmoid', name='OuputLayer'))
    return model
arch().summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 HiddenLayer1 (Dense)        (None, 8)                 32        
                                                                 
 HiddenLayer2 (Dense)        (None, 4)                 36        
                                                                 
 OuputLayer (Dense)          (None, 1)                 5         
                                                                 
Total params: 73 (292.00 Byte)
Trainable params: 73 (292.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
def bceloss(y_true, y_pred):
    # Clip the prediction value to prevent log(0) error
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Calculate the weighted binary cross entropy loss
    loss = -(0.1 * (y_true * K.log(y_pred)) + 0.9*((1 - y_true) * K.log(1 - y_pred)))
    return K.mean(loss, axis=-1)

model=arch()
model.summary
opt=SGD( learning_rate=0.01, momentum=0.9)
model.compile(loss=binary_crossentropy, optimizer='adam', metrics=['accuracy'])
model_path="../Model/NN2/tmp/{epoch:02d}-{val_accuracy:.4f}.h5"
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
H=model.fit(x_train,y_train,
          validation_split=0.2,
          epochs=100,batch_size=64,
          callbacks=callbacks_list,
          verbose=1)

In [94]:
model=load_model('../Model/NN2/Best/64-0.9771.h5', compile=False)
y_pred=model.predict(x_test)
count=0
count1=0
for i in range(y_pred.shape[0]):
    if y_test[i]==0:
        count+=1
        if y_pred[i]<=0.5:
            count1+=1
print(count,count1)
count=0
count1=0
for i in range(y_pred.shape[0]):
    if y_test[i]==1:
        count+=1
        if y_pred[i]>=0.5:
            count1+=1
print(count,count1)

24/24 [==============================] - 0s 578us/step
388 338
375 336


## Multichannel

In [ ]:
state0=np.load('../Data/new/2gsps/Q1state0_her.npy')
state1=np.load('../Data/new/2gsps/Q1state1_her.npy')